## Imports

In [4]:
import sys
from pathlib import Path

module_path = str(Path("../src/data").resolve())
if module_path not in sys.path:
    sys.path.append(module_path)


In [5]:
sys.path

['/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python313.zip',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/lib-dynload',
 '',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/site-packages',
 '/home/lestrada/tumor_type_prediction',
 '/home/lestrada/tumor_type_prediction/src/data']

In [16]:
import LogRegFxF as LR
import preprocessing as prep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, kstest
from scipy import stats  

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV,ElasticNetCV, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, classification_report, f1_score, matthews_corrcoef, mean_squared_error,r2_score, roc_auc_score, roc_curve, auc, confusion_matrix, log_loss
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import kstest_normal
from timeit import default_timer as timer
from tqdm import tqdm  
from typing import Optional
from joblib import Parallel, delayed
import pickle

In [21]:
import importlib
importlib.reload(prep)
dir(prep)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'impute_normal_down_shift_distribution',
 'np',
 'pd',
 'post_process_meta_intensities']

In [17]:
dir(prep)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'pd',
 'post_process_meta_intensities']

## Data Import

In [10]:
#Proteins quantification intensities file
processed_data = '2024.10.23_CJ_pancancer_250/'
folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
PREPROCESSED_FP_INTENSITY = 'preprocessed_fp_with_ref.csv'
intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
input_quantifications = pd.read_csv(intensity_path_file)

#--------------------------------------------------------------------------------

#Samples metadata (oncotree classification) file.
metadata_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_MTBs_Evaluation/'
metadata_file = 'METADATA_PAN_CANCER_Batch300.xlsx'
the_metadata_file = metadata_path + metadata_file
input_metadata = pd.read_excel(the_metadata_file)

#--------------------------------------------------------------------------------

# # Proteins quantification z-scores file
# processed_data = '2024.10.23_CJ_pancancer_250/'
# folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
# PREPROCESSED_FP_INTENSITY = 'full_proteome_measures_z.tsv'
# intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
# df_Z_scores = pd.read_csv(intensity_path_file, sep='\t')

# Data Preprocessing

In [ ]:
# Protein quantification intensities post-processing
input_quantifications = input_quantifications.set_index(input_quantifications.columns[0])
peptides_quant_info = prep.post_process_meta_intensities(input_quantifications.iloc[:,int(input_quantifications.shape[1]/2):].T ) #clean dataframe from regex characers
proteins_quant = input_quantifications.iloc[:,:int(input_quantifications.shape[1]/2)].T #subset protein measurements from dataset

#Imputation
prot_quant_imputed = prep.impute_normal_down_shift_distribution(proteins_quant) #Imputation of missing values in protein intensities using normal distribution down-shift method
na_columns = prot_quant_imputed.isna().any()
na_columns_true = na_columns[na_columns].index.tolist()
print("Proteins with  empty values:", na_columns_true)

#Cleaning sample names
prot_quant_imputed.reset_index(inplace=True)
prot_quant_imputed.rename(columns={'index': 'Sample name'}, inplace=True)
prot_quant_imputed['Sample name'] = prot_quant_imputed['Sample name'].str.replace('pat_', '')

#Dataset with protein intensities and metadata
samples_metadata = input_metadata[["Sample name", "code_oncotree",]] #sample metadata e.g. class, TCC, tissue of origin, etc.
initial_df = samples_metadata.merge(prot_quant_imputed, left_on='Sample name', right_on='Sample name')

# Data Set Split